In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.join(os.path.dirname('/home/dhkim/OpenSetGNN-main/src/model')))
sys.path.append(os.path.join(os.path.dirname('/home/dhkim/OpenSetGNN-main/src/utils')))

In [2]:
import torch
import torchvision
import torch_geometric
import torch_scatter
import torch_sparse
import torch_cluster

print(f"PyTorch Version: {torch.__version__}")
print(f"PyTorch CUDA Version: {torch.version.cuda}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"TorchVision Version: {torchvision.__version__}")
print(f"PyTorch Geometric Version: {torch_geometric.__version__}")
print(f"Torch-scatter Version: {torch_scatter.__version__}")
print(f"Torch-sparse Version: {torch_sparse.__version__}")
print(f"Torch-cluster Version: {torch_cluster.__version__}")

# Simple CUDA test
if torch.cuda.is_available():
    x = torch.randn(5, 5).cuda()
    print(f"Tensor on CUDA: {x.is_cuda}")
else:
    print("CUDA is not available.")

/home/dhkim/pyg_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch Version: 1.12.0a0+2c916ef.nv22.3
PyTorch CUDA Version: 11.4
CUDA available: True
TorchVision Version: 0.13.0a0+da3794e
PyTorch Geometric Version: 2.1.0
Torch-scatter Version: 2.1.1
Torch-sparse Version: 0.6.15
Torch-cluster Version: 1.6.1
Tensor on CUDA: True


In [3]:
from model.GAE_Projection_Att import GAE_CLS_Link_NODE_Cosine_SupCon_2
from model.resnet_big import SupCEResNet, SupConResNet, LinearClassifier, SupIncepResnet
import torch
# from torch_geometric.nn import summary
from thop import profile
import numpy as np
import time
from torch_geometric.loader import DataLoader

from torch.profiler import profile, record_function, ProfilerActivity
import torch.nn.functional as F
from thop import profile as thopprofile, clever_format
from torch_geometric.utils import to_dense_adj

In [4]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [5]:
def measure_time_gpu(dummy_input, model, device, rep, none_gnn=False):
    model = model.to(device=device)
    # dummy_input = torch.randn(1, 1, 29, 29, dtype=torch.float).to(device)
    # INIT LOGGERS
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    repetitions = rep
    timings=np.zeros((repetitions,1))
    #GPU-WARM-UP
    for _ in range(100):
        if (none_gnn):
            _ = model(dummy_input)
        else:
            _ = model(dummy_input, device, acummulate = True, remove_random=True)
    # MEASURE PERFORMANCE
    with torch.no_grad():
        for rep in range(repetitions):
            starter.record()
            if (none_gnn):
                _ = model(dummy_input)
            else:
                _ = model(dummy_input, device, acummulate = True, remove_random=True)
            ender.record()
            # WAIT FOR GPU SYNC
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)
            timings[rep] = curr_time
    mean_syn = np.sum(timings) / repetitions
    std_syn = np.std(timings)
    return mean_syn, std_syn

In [6]:
def measure_time_cpu(model, device, rep = 10):
    model = model.to(device=device)
    x = torch.rand((1, 1, 29, 29), device=device)
    timings=np.zeros((rep,1))
    for i in range(rep):    
        start_time = time.time()
        out = model(x)
        timings[i] = time.time() - start_time
    mean_syn = np.sum(timings) / rep
    std_syn = np.std(timings)
    return mean_syn, std_syn

In [7]:
def calculate_graph_reconstruction_loss(reconstructed, original, batch_index, reduced="mean"):
    """
    Calculate the mean reconstruction loss for each graph in a batch.
    
    Args:
        reconstructed (torch.Tensor): Reconstructed node features, shape [num_nodes, num_features].
        original (torch.Tensor): Original node features, shape [num_nodes, num_features].
        batch_index (torch.Tensor): Batch indices mapping each node to a graph, shape [num_nodes].
        
    Returns:
        torch.Tensor: Mean reconstruction loss for each graph, shape [num_graphs].
    """
    # Calculate MSE loss for each node
    node_mse_loss = torch.mean((reconstructed - original) ** 2, dim=1)  # Shape: [num_nodes]

    # Aggregate losses for each graph
    num_graphs = batch_index.max().item() + 1  # Total number of graphs in the batch
    graph_losses = torch.zeros(num_graphs, device=reconstructed.device)  # Initialize graph loss storage

    for graph_id in range(num_graphs):
        # Mask to select nodes belonging to the current graph
        graph_mask = (batch_index == graph_id)
        
        # Mean MSE loss for the current graph
        if (reduced == "mean"):
            graph_losses[graph_id] = torch.mean(node_mse_loss[graph_mask])
        elif (reduced == "none"):
            return graph_losses
        else:
            graph_losses[graph_id] = sum(node_mse_loss[graph_mask])
    return graph_losses

def open_set_inference_optimized(ae_model,
                                 batch,
                                 thresholds,
                                 class_centers=None,
                                 device='cuda',
                                 num_known_classes=None): # Add num_known_classes
    """
    Optimized forward pass for open-set inference.
    """
    ae_model.eval()
    batch = batch.to(device)

    # --- Model Forward Pass (No changes here, assumed to be optimized within ae_model) ---
    with torch.no_grad():
        M_sup = ae_model(batch, device, acummulate=True, remove_random=True)
        graph_emb_sup = ae_model.graph_embedding(M_sup, batch.batch)
        graph_emb_sup = F.normalize(graph_emb_sup, p=2, dim=1) # [B, EmbDim]

        # Calculate distances to all class centers
        dists = torch.cdist(graph_emb_sup, class_centers, p=2) # [B, C_known]
        logits = -dists
        probs = F.softmax(logits, dim=1) # [B, C_known]
        max_probs, pred_cls_known = probs.max(dim=1) # max_probs and indices for known classes

        # Reconstruction
        # Adjacency reconstruction loss (graph-level)
        adj_rec = ae_model.adj_decode(M_sup, batch.batch, use_sigmoid=False)
        adj_ori = to_dense_adj(batch.edge_index,
                               batch.batch,
                               edge_attr=batch.edge_attr[:, 0].unsqueeze(1) if batch.edge_attr is not None else None).squeeze(3)
        # Ensure adj_rec and adj_ori have compatible shapes for mse_loss
        # This might require padding or careful handling if graph sizes vary significantly
        # and ae_model.adj_decode doesn't produce consistently sized outputs per graph
        # For now, assuming they are compatible or loss handles it.
        adj_loss = F.mse_loss(adj_rec, adj_ori, reduction='none').sum(dim=(1, 2)) # [B]

        # Node reconstruction loss (graph-level)
        node_hat = ae_model.node_recon(M_sup)
        node_rec_loss = calculate_graph_reconstruction_loss(node_hat, batch.x, batch.batch, reduced="sum") # [B]

    # --- Open-Set Logic (Vectorized) ---
    num_graphs_in_batch = graph_emb_sup.size(0)
    if num_known_classes is None and class_centers is not None:
        num_known_classes = class_centers.size(0)
    elif num_known_classes is None:
        raise ValueError("num_known_classes must be provided if class_centers is None or for unknown label assignment")

    # Initialize predictions as the predicted known class
    pred_out = pred_cls_known.clone() # [B]

    # Rule 1: Distance to predicted class center > its threshold (vectorized)
    # Gather the specific distance thresholds for each predicted class
    # thresholds['distance'] is expected to be a tensor or list of length C_known
    distance_thresholds_for_pred_cls = torch.tensor(thresholds['distance'], device=device)[pred_cls_known] # [B]
    distances_to_pred_cls = dists.gather(1, pred_cls_known.unsqueeze(1)).squeeze(1) # [B]
    unknown_due_to_distance_to_pred = (distances_to_pred_cls > distance_thresholds_for_pred_cls) # [B]

    # Initialize reasons (0: benign, 1: known_attack, >1: unknown due to specific anomaly)
    reasons = torch.zeros(num_graphs_in_batch, dtype=torch.long, device=device)
    reasons[pred_cls_known > 0] = 1 # Mark known attacks (assuming class 0 is benign)

    # SVDD distance for benign class (class 0)
    d_svdd_benign = dists[:, 0] # [B]

    # Identify samples initially predicted as benign (class 0) OR
    # those that already violated their predicted class's distance threshold
    # These are candidates for being re-classified as 'unknown' based on anomaly metrics
    # or staying benign if they pass all checks.
    benign_candidates_mask = (pred_cls_known == 0) | unknown_due_to_distance_to_pred # [B]

    # --- Anomaly checks for benign_candidates_mask ---
    # These flags are only relevant for samples in benign_candidates_mask
    adj_flag_all = (adj_loss > thresholds['adj']) # [B]
    node_flag_all = (node_rec_loss > thresholds['node']) # [B]
    svdd_flag_all = (d_svdd_benign > thresholds['distance'][0]) # [B], distance to benign center threshold

    # Combine flags for reason codes (only for benign candidates)
    # We apply the benign_candidates_mask *after* calculating all flags to keep indexing simple
    adj_flag_bc = adj_flag_all[benign_candidates_mask]
    node_flag_bc = node_flag_all[benign_candidates_mask]
    svdd_flag_bc = svdd_flag_all[benign_candidates_mask]

    # Calculate reason codes for the benign candidates subset
    flag_combo_bc = (adj_flag_bc.long() * 1 +
                     node_flag_bc.long() * 2 +
                     svdd_flag_bc.long() * 4) # yields 0-7 for benign candidates

    # Map combo to final reason codes (2-8 for anomalies, 0 if still benign)
    # combo_to_reason_map_tensor should be precomputed and on the correct device
    # For example:
    # _combo_map = {0:0, 1:2, 2:3, 3:5, 4:4, 5:6, 6:7, 7:8}     0 means stays benign/known attack
    # combo_to_reason_map_tensor = torch.tensor([_combo_map[i] for i in range(8)], device=device)
    # Optimized: Create this map directly
    combo_to_reason_map_tensor = torch.tensor([0, 2, 3, 5, 4, 6, 7, 8], device=device, dtype=torch.long)
    reasons_for_benign_candidates = combo_to_reason_map_tensor[flag_combo_bc]

    # Update reasons for those initially benign candidates
    # If reasons_for_benign_candidates > 0, it means it's an anomaly type
    reasons[benign_candidates_mask] = torch.where(
        reasons_for_benign_candidates > 0, # If an anomaly reason code was generated
        reasons_for_benign_candidates,     # Use that anomaly reason
        reasons[benign_candidates_mask]    # Otherwise, keep original reason (0 for benign, 1 for known attack if it was a misclassified known that passed distance)
    )

    # Samples become 'unknown' if:
    # 1. They were benign_candidates AND any of their anomaly flags (adj, node, svdd for benign) were true
    # 2. OR they initially violated their predicted class's distance threshold (unknown_due_to_distance_to_pred)
    is_unknown_anomaly = (benign_candidates_mask & ( (adj_flag_all | node_flag_all | svdd_flag_all)[benign_candidates_mask] )) \
                       | unknown_due_to_distance_to_pred

    pred_out[is_unknown_anomaly] = num_known_classes # Assign 'unknown' label (e.g., C_known if labels are 0 to C_known-1)

    # Ensure pred_out for samples that are not unknown, but were initially marked as known attacks (>0),
    # and did NOT violate distance_to_pred, retain their known attack label.
    # This is implicitly handled as pred_out is initialized with pred_cls_known.
    # We only overwrite with 'unknown' or keep benign/known_attack.

    return pred_out.cpu().numpy(), reasons.cpu().numpy(), \
           d_svdd_benign.cpu(), adj_loss.cpu(), node_rec_loss.cpu(), \
           graph_emb_sup # graph_emb_sup is already on device, return as is or move to cpu() if needed by caller

In [83]:
def open_set_inference_phase_0(ae_model,
                       batch,
                       thresholds,
                       class_centers=None,
                       device='cuda',
                       num_classes = 5):
    M_sup = ae_model(batch, device, acummulate=True, remove_random=True)
#     graph_emb_sup = ae_model.graph_embedding(M_sup, batch.batch)
#     graph_emb_sup = F.normalize(graph_emb_sup, p=2, dim=1)

# #         # Distance to class centers and classification
# #         # For batch_size=1, dists will be [1, C_known]
#     dists = torch.cdist(graph_emb_sup, class_centers, p=2)
#     logits = -dists # Negative distance can be used as logits for closest class
#     probs = F.softmax(logits, dim=1) # Softmax over known classes [1, C_known]
#     _, pred_cls = probs.max(dim=1) # Predicted class for the single graph [1]

#         # Reconstruction (recon) calculations
#         # adj_rec will be [1, N, N]
#     adj_rec = ae_model.adj_decode_optimized(M_sup, batch.batch, use_sigmoid=False)
# #         # node_hat is assumed to be a tensor of node-wise errors for the single graph
#     node_hat = ae_model.node_recon(M_sup)
# #         # adj_ori will be [1, N, N]
#     adj_ori = to_dense_adj(batch.edge_index,
#                                batch.batch,
#                                edge_attr=batch.edge_attr[:, 0].unsqueeze(1)).squeeze(3)

#         # Calculate reconstruction losses
#         # Sum over spatial dimensions for adjacency loss for the single graph [1]
#         adj_loss = F.mse_loss(adj_rec, adj_ori, reduction='none').sum(dim=(1, 2))
#         # Sum over all node errors for node reconstruction loss for the single graph [1]
#         rec_loss = node_hat.sum(dim=-1).unsqueeze(0) # Ensure it's [1] for batch_size=1 consistent with adj_loss

#     # 1) For each sample, find the closest class center and check against its threshold
#     pred_out = pred_cls.cpu().numpy()              # 0 = benign, >0 = known attack

#     unknown_distance = torch.tensor([False]).to(device)
#     distance = dists[0, pred_out[0]]
#     if (distance > thresholds['distance'][pred_out[0]]):
#         unknown_distance[0] = True

    return M_sup

In [84]:
def open_set_inference_phase_1(ae_model,
                       batch,
                       thresholds,
                       class_centers=None,
                       device='cuda',
                       num_classes = 5):
    M_sup = ae_model(batch, device, acummulate=True, remove_random=True)
    graph_emb_sup = ae_model.graph_embedding(M_sup, batch.batch)
    graph_emb_sup = F.normalize(graph_emb_sup, p=2, dim=1)

#         # Distance to class centers and classification
#         # For batch_size=1, dists will be [1, C_known]
    dists = torch.cdist(graph_emb_sup, class_centers, p=2)
    logits = -dists # Negative distance can be used as logits for closest class
    probs = F.softmax(logits, dim=1) # Softmax over known classes [1, C_known]
    _, pred_cls = probs.max(dim=1) # Predicted class for the single graph [1]

#         # Reconstruction (recon) calculations
#         # adj_rec will be [1, N, N]
#     adj_rec = ae_model.adj_decode_optimized(M_sup, batch.batch, use_sigmoid=False)
# #         # node_hat is assumed to be a tensor of node-wise errors for the single graph
#     node_hat = ae_model.node_recon(M_sup)
# #         # adj_ori will be [1, N, N]
#     adj_ori = to_dense_adj(batch.edge_index,
#                                batch.batch,
#                                edge_attr=batch.edge_attr[:, 0].unsqueeze(1)).squeeze(3)

#         # Calculate reconstruction losses
#         # Sum over spatial dimensions for adjacency loss for the single graph [1]
#         adj_loss = F.mse_loss(adj_rec, adj_ori, reduction='none').sum(dim=(1, 2))
#         # Sum over all node errors for node reconstruction loss for the single graph [1]
#         rec_loss = node_hat.sum(dim=-1).unsqueeze(0) # Ensure it's [1] for batch_size=1 consistent with adj_loss

#     # 1) For each sample, find the closest class center and check against its threshold
    pred_out = pred_cls.cpu().numpy()              # 0 = benign, >0 = known attack

    unknown_distance = torch.tensor([False]).to(device)
    distance = dists[0, pred_out[0]]
    if (distance > thresholds['distance'][pred_out[0]]):
        unknown_distance[0] = True

    return M_sup

In [85]:
def open_set_inference_phase_2(ae_model,
                       batch,
                       thresholds,
                       class_centers=None,
                       device='cuda',
                       num_classes = 5):
    M_sup = ae_model(batch, device, acummulate=True, remove_random=True)
#     graph_emb_sup = ae_model.graph_embedding(M_sup, batch.batch)
#     graph_emb_sup = F.normalize(graph_emb_sup, p=2, dim=1)

# #         # Distance to class centers and classification
# #         # For batch_size=1, dists will be [1, C_known]
#     dists = torch.cdist(graph_emb_sup, class_centers, p=2)
#     logits = -dists # Negative distance can be used as logits for closest class
#     probs = F.softmax(logits, dim=1) # Softmax over known classes [1, C_known]
#     _, pred_cls = probs.max(dim=1) # Predicted class for the single graph [1]

        # Reconstruction (recon) calculations
        # adj_rec will be [1, N, N]
    adj_rec = ae_model.adj_decode_optimized(M_sup, batch.batch, use_sigmoid=False)
#         # node_hat is assumed to be a tensor of node-wise errors for the single graph
    node_hat = ae_model.node_recon(M_sup)
#         # adj_ori will be [1, N, N]
    adj_ori = to_dense_adj(batch.edge_index,
                               batch.batch,
                               edge_attr=batch.edge_attr[:, 0].unsqueeze(1)).squeeze(3)

    # Calculate reconstruction losses
    # Sum over spatial dimensions for adjacency loss for the single graph [1]
    adj_loss = F.mse_loss(adj_rec, adj_ori, reduction='none').sum(dim=(1, 2))
    # Sum over all node errors for node reconstruction loss for the single graph [1]
    rec_loss = node_hat.sum(dim=-1).unsqueeze(0) # Ensure it's [1] for batch_size=1 consistent with adj_loss

    # 1) For each sample, find the closest class center and check against its threshold
    # pred_out = pred_cls.cpu().numpy()              # 0 = benign, >0 = known attack

    # unknown_distance = torch.tensor([False]).to(device)
    # distance = dists[0, pred_out[0]]
    # if (distance > thresholds['distance'][pred_out[0]]):
    #     unknown_distance[0] = True

    return M_sup

In [91]:
import time
import numpy as np
import torch

def analyze_performance_time_counter(model, test_loader, device='cuda', num_warmup=10, num_repeats=100, num_classes=5):
    model = model.to(device)
    model.eval()
    times, times_1, times_2 = [], [], []

    with torch.no_grad():
        # Warm-up
        print(f"Warming up for {num_warmup} iterations...")
        for i, batch in enumerate(test_loader):
            if i >= num_warmup:
                break
            batch = batch.to(device)
            _ = model(batch) # Assuming model call is the main operation

        print(f"Starting timed inference over {num_repeats} iterations...")
        for i, batch in enumerate(test_loader):
            if i >= num_repeats:
                break

            batch = batch.to(device)
            
            # Ensure GPU is idle before starting the timer
            torch.cuda.synchronize()
            start_time = time.perf_counter()

            open_set_inference_phase_0(model, 
                                 batch, 
                                 thresholds={'distance': [0.5]*num_classes, 'adj': 0.1, 'node': 0.1}, 
                                 device=device,
                                 class_centers=torch.zeros((num_classes, 128), device=device), 
                                 num_classes=num_classes)
            
            # Wait for the operation to complete
            torch.cuda.synchronize()
            
            end_time = time.perf_counter()
            
            # Time is in seconds, convert to milliseconds
            times.append((end_time - start_time) * 1000)
            
            # Ensure GPU is idle before starting the timer
            torch.cuda.synchronize()
            start_time = time.perf_counter()

            open_set_inference_phase_1(model, 
                                 batch, 
                                 thresholds={'distance': [0.5]*num_classes, 'adj': 0.1, 'node': 0.1}, 
                                 device=device,
                                 class_centers=torch.zeros((num_classes, 128), device=device), 
                                 num_classes=num_classes)
            
            # Wait for the operation to complete
            torch.cuda.synchronize()
            
            end_time = time.perf_counter()
            
            # Time is in seconds, convert to milliseconds
            times_1.append((end_time - start_time) * 1000)
            
            # Ensure GPU is idle before starting the timer
            torch.cuda.synchronize()
            start_time = time.perf_counter()

            open_set_inference_phase_2(model, 
                                 batch, 
                                 thresholds={'distance': [0.5]*num_classes, 'adj': 0.1, 'node': 0.1}, 
                                 device=device,
                                 class_centers=torch.zeros((num_classes, 128), device=device), 
                                 num_classes=num_classes)
            
            # Wait for the operation to complete
            torch.cuda.synchronize()
            
            end_time = time.perf_counter()
            
            # Time is in seconds, convert to milliseconds
            times_2.append((end_time - start_time) * 1000)

        times = np.array(times)
        times_1 = np.array(times_1)
        times_2 = np.array(times_2)
        
        mean_enc = np.mean(times)
        med_enc = np.median(times)
        throughput_enc = 1000 / np.mean(times)
        print(f"Tested {len(times)} samples.")
        print(f"Mean Latency Encoding: {np.mean(times):.2f} ms")
        print(f"Median Latency Encoding: {np.median(times):.2f} ms")
        print(f"Throughput Encoding: {1000 / np.mean(times):.2f} samples/sec")
        
        print(f"Mean Latency Classification: {(np.mean(times_1) - mean_enc):.2f} ms")
        print(f"Median Latency Classification: {(np.median(times_1) - med_enc):.2f} ms")
        print(f"Throughput Classification: {1000 / (np.mean(times_1)):.2f} samples/sec")
        
        print(f"Mean Latency Anomaly Detection: {(np.mean(times_2) - mean_enc):.2f} ms")
        print(f"Median Latency Anomaly Detection: {(np.median(times_2) - med_enc):.2f} ms")
        print(f"Throughput Anomaly Detection: {1000 / (np.mean(times_2)):.2f} samples/sec")

In [73]:
import torch
import os
import random
from typing import List, Tuple, Any

def load_and_split_graphs(
    path: str,
    exclude: List[str] = None,
    train_ratio: float = 0.8,
    seed: int = 2025
) -> Tuple[List[Any], List[Any], List[str]]:
    """
    Loads graph data from .pt files in a specified directory, excludes certain graphs,
    and splits them into training and testing sets.

    Args:
        path (str): The directory containing the .pt graph files.
        exclude (List[str], optional): A list of graph filenames (without .pt extension)
                                       to exclude from loading. Defaults to None.
        train_ratio (float, optional): The proportion of graphs to allocate to the
                                       training set. Should be between 0.0 and 1.0.
                                       Defaults to 0.8.
        seed (int, optional): The random seed for reproducibility of the split.
                              Defaults to 2025.

    Returns:
        Tuple[List[Any], List[Any], List[str]]: A tuple containing:
            - train_graphs (List[Any]): List of graph objects for the training set.
            - test_graphs (List[Any]): List of graph objects for the testing set.
            - graphs_names (List[str]): List of all loaded graph filenames (base names,
                                        without .pt extension and excluding excluded graphs).
    """
    if exclude is None:
        exclude = []

    all_graphs_with_names = []
    loaded_graph_names = []

    # Ensure the path exists
    if not os.path.isdir(path):
        print(f"Error: Directory not found at '{path}'")
        return [], [], []

    # Iterate through files in the directory
    for filename in os.listdir(path):
        if filename.endswith(".pt"):
            graph_name = os.path.splitext(filename)[0] # Get name without .pt extension

            if graph_name in exclude:
                print(f"Skipping excluded graph: {filename}")
                continue

            filepath = os.path.join(path, filename)
            try:
                # Load the graph object
                graph_data = torch.load(filepath)
                all_graphs_with_names.extend(graph_data)
                loaded_graph_names.append(graph_name)
                print(f"Loaded graph: {filename}")
            except Exception as e:
                print(f"Error loading {filename}: {e}")
                continue

    if not all_graphs_with_names:
        print("No graphs loaded or all graphs were excluded.")
        return [], [], []
    
    print(all_graphs_with_names[0])


    # Split into train and test sets
    random.seed(seed)
    random.shuffle(all_graphs_with_names) # Shuffle the combined list for random split

    num_train = int(len(all_graphs_with_names) * train_ratio)

    train_data = all_graphs_with_names[:num_train]
    test_data = all_graphs_with_names[num_train:]

    train_graphs = [item for item in train_data]
    test_graphs = [item for item in test_data]
    
    # Return the names of the graphs that were actually loaded and split
    # For clarity, we'll return the names of the graphs that ended up in train/test
    # and the original list of all loaded names (before split).
    
    return train_graphs, test_graphs, loaded_graph_names


In [11]:
path = f'/home/dhkim/OpenSetGNN-main/data'

train_graphs, test_graphs, graphs_names = load_and_split_graphs(path, exclude=[], train_ratio=0.8, seed=2025)

Loaded graph: reverse_light_on_graphs.pt
Loaded graph: correlated_signal_graphs.pt
Loaded graph: reverse_light_off_graphs.pt
Loaded graph: max_speedometer_graphs.pt
Loaded graph: max_engine_coolant_temp_graphs.pt
Data(x=[57, 9], edge_index=[2, 128], edge_attr=[128, 3], y=[1], node_labels=[57])


In [12]:
# Load data
warmup_loader = DataLoader([train_graphs[0]], batch_size=1, shuffle=False)

for graph in test_graphs:
    graph.edge_attr = graph.edge_attr[:, 0].reshape(-1, 1).clone().detach()

test_loader = DataLoader(test_graphs, batch_size=1, shuffle=False)

dummy_input = next(iter(warmup_loader)).to('cuda')
print(dummy_input)
dummy_input.edge_attr = dummy_input.edge_attr[:, 0].reshape(-1, 1).clone().detach()

DataBatch(x=[69, 9], edge_index=[2, 130], edge_attr=[130, 3], y=[1], node_labels=[69], batch=[69], ptr=[2])


In [13]:
len(train_graphs)

4681

In [14]:
NUM_CLASSES = len(np.unique([g.y.item() for g in train_graphs]))
NUM_CLASSES

6

In [37]:
model = GAE_CLS_Link_NODE_Cosine_SupCon_2(num_features=9, embedding_size=32, projection_emb=128, activate='gelu', layer_type='gatv2', num_layers=2, directed=False, id_dim=1, num_classes = NUM_CLASSES, linear_node=True, num_id_embeddings=2048, attn_head=1)
measure_time_gpu(dummy_input, model, 'cuda', rep=1000)

(6.99295936536789, 0.5312164604079275)

In [16]:
incep = SupIncepResnet(num_classes=NUM_CLASSES)
measure_time_gpu(torch.randn(1, 1, 29, 29, dtype=torch.float).cuda(), incep, 'cuda', rep=1000, none_gnn=True)

(6.1718022465705875, 0.6611099079180738)

In [17]:
supcon = SupCEResNet(name='resnet18', num_classes=NUM_CLASSES)
measure_time_gpu(torch.randn(1, 1, 29, 29, dtype=torch.float).cuda(), supcon, 'cuda', rep=1000, none_gnn=True)

(6.0121413507461545, 0.7200512861885778)

In [92]:
analyze_performance_time_counter(model, test_loader, device='cuda', num_warmup=10, num_repeats=100, num_classes=NUM_CLASSES)

Warming up for 10 iterations...
Starting timed inference over 100 iterations...
Tested 100 samples.
Mean Latency Encoding: 7.88 ms
Median Latency Encoding: 6.81 ms
Throughput Encoding: 126.93 samples/sec
Mean Latency Classification: 4.40 ms
Median Latency Classification: 3.97 ms
Throughput Classification: 81.41 samples/sec
Mean Latency Anomaly Detection: 2.85 ms
Median Latency Anomaly Detection: 2.29 ms
Throughput Anomaly Detection: 93.19 samples/sec


In [76]:
analyze_performance_profiler(model, test_loader, device='cuda')

-------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                            model_inference        43.42%     158.073ms        99.95%     363.871ms     363.871ms     163.375ms        44.89%     363.845ms     363.845ms          -4 b        -444 b           0 b      -5.78 Mb             1  
                              

In [19]:
def profile_gnn_model(model, example_data, device=None, repeat=10):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device).eval()
    example_data = example_data.to(device)
    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                 record_shapes=True,
                 profile_memory=True,
                 with_flops=True) as prof:
        with torch.no_grad():
            for _ in range(repeat):
                with record_function("model_inference"):
                    model(example_data)
    print(prof.key_averages().table(sort_by="self_cuda_time_total", row_limit=15))
    print(prof.key_averages().table(sort_by="flops", row_limit=15))
    # For total FLOPs (note: double to get MACs for real-valued ops)
    total_flops = sum([item.flops for item in prof.key_averages() if hasattr(item, 'flops')])
    print(f'Estimated total FLOPs: {clever_format([total_flops], "%.3f")}')
    print(f'Estimated total MACs: {(total_flops/2)} {clever_format([total_flops/2], "%.3f")}')

In [20]:
profile_gnn_model(supcon.to(device='cpu'), torch.randn(1, 1, 29, 29, dtype=torch.float).cuda())

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Total KFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                 model_inference        23.44%      39.046ms        99.48%     165.712ms      16.571ms      40.154ms        24.11%     165.704ms      16.570ms         -40 b      -4.34 Kb           0 b     -14.95 Mb            10            --  
         aten::cud

In [21]:
dummy_input = dummy_input.to(device='cpu')
model = model.to(device='cpu')
profile_gnn_model(model, dummy_input)

-------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Total KFLOPs  
-------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                            model_inference        38.59%     125.039ms        99.77%     323.262ms      32.326ms     131.166ms        40.49%     323.252ms      32.325ms     271.14 Kb    -126.84 Kb    -274.00 Kb      -5.57 Mb     

In [22]:
macs, params = thopprofile(incep, inputs=(torch.randn(1, 1, 29, 29, dtype=torch.float).cuda(),), verbose=False)
macs, params = clever_format([macs, params], "%.3f")
print(f"MACs: {macs}") # thop reports MACs
print(f"Params: {params}")

MACs: 97.191M
Params: 1.695M


In [23]:
macs, params = thopprofile(supcon, inputs=(torch.randn(1, 1, 29, 29, dtype=torch.float).cuda(),), verbose=False)
macs, params = clever_format([macs, params], "%.3f")
print(f"MACs: {macs}") # thop reports MACs
print(f"Params: {params}")

MACs: 32.561M
Params: 700.662K


In [24]:
macs, params = thopprofile(model, inputs=(dummy_input,), verbose=False)

print(f"MACs: {macs}") # thop reports MACs
print(f"Params: {params}")

MACs: 0.0
Params: 0.0
